In [1]:
import random
import math
import import_ipynb
from WordRAM import CreateMachine, Reg, Mem, GetMachine

importing Jupyter notebook from WordRAM.ipynb


In [2]:
class BitVector:
    def __init__(self, n, verbose=False):
        self.n = n
        self.verbose = verbose
        lgn = Reg(n).lgceil()
        assert lgn._x * lgn._x < n, "The BitVector is too small."
        # allocate one more slot for better readability
        self.bs = Mem(Reg(3), lgn) # block sizes of rank index
        self.rwl = Mem(Reg(4), lgn) # word lengths of rank index
        self.rsize = Mem(Reg(4), lgn) # size of rank index
        self.bs[Reg(1)] = lgn * lgn # block size of r1
        self.bs[Reg(2)] = lgn / Reg(2) # block size of r2
        if verbose: print("bs: " + str(self.bs))
        self.rwl[Reg(1)] = lgn # at most n ones
        self.rwl[Reg(2)] = (self.bs[Reg(1)] + Reg(1)).lgceil() # at most bs1 ones
        self.rwl[Reg(3)] = (self.bs[Reg(2)] + Reg(1)).lgceil() # at most bs2 ones
        if verbose: print("rwl: " + str(self.rwl))
        self.rsize[Reg(1)] = Reg(n).divceil(self.bs[Reg(1)])
        self.rsize[Reg(2)] = self.rsize[Reg(1)] * (self.bs[Reg(1)].divceil(self.bs[Reg(2)]))
        self.rsize[Reg(3)] = (Reg(1) << self.bs[Reg(2)]) * self.bs[Reg(2)] # at most 2 ** `bs[2]` rows and `bs[2]` columns
        if verbose: print("rsize: " + str(self.rsize))
            
        self._bv = Mem(Reg(n), Reg(1))
        
    def _randomize(self):
        # Generate data randomly for experiments
        self._bv._randomize()
        
    def buildIndex(self):
        # Build brute force index
        # Only build index for rank1, because rank0 can be easily derived from rank1
        self.indexSizeBF = 0
        self._r1bf = [] # brute force index for rank1
        crt = 0
        for i in range(self.n):
            crt += self._bv._getBit(i) == 1
            self._r1bf.append(crt)
        self.indexSizeBF += len(self._r1bf) * math.ceil(math.log(self.n + 1, 2))
        
        self._s0bf = [None for _ in range(self.n)] # brute force index for select0
        self._s1bf = [None for _ in range(self.n)] # brute force index for select1
        crt0 = crt1 = 0
        for i in range(self.n):
            b = self._bv._getBit(i)
            if b == 0:
                self._s0bf[crt0] = i
                crt0 += 1
            elif b == 1:
                self._s1bf[crt1] = i
                crt1 += 1
        
        # Build succinct index
        # Only build index for rank1, because rank0 can be easily derived from rank1
        self._r1 = Mem(self.rsize[Reg(1)], self.rwl[Reg(1)])
        self._r2 = Mem(self.rsize[Reg(2)], self.rwl[Reg(2)])
        self._r3 = Mem(self.rsize[Reg(3)], self.rwl[Reg(3)])
        c1 = Reg(0) # c1 ones in the whole vector
        c2 = Reg(0) # c2 ones in some large block
        i = Reg(0) # i-th bit in the whole vector
        j = Reg(0) # j-th bit in some large block
        k = Reg(0) # k-th small block
        n = Reg(self.n)
        while i < n:
            if i % self.bs[Reg(1)] == Reg(0):
                self._r1[i / self.bs[Reg(1)]] = c1
                c2 = Reg(0)
                j = Reg(0)
            if j % self.bs[Reg(2)] == Reg(0):
                self._r2[k] = c2
                k += Reg(1)
            
            b = self._bv._getBit(i._x)
            if (b == 1):
                c1 += Reg(1)
                c2 += Reg(1)
            
            i += Reg(1)
            j += Reg(1)
            
        row = Reg(1) << self.bs[Reg(2)]
        column = self.bs[Reg(2)]
        i = Reg(0)
        while i < row:
            j = Reg(0)
            while j < column:
                self._r3[i * column + j] = (i >> (column - j - Reg(1))).getOnes()
                j += Reg(1)
            i += Reg(1)
        if self.verbose:
            print("r1: " + str(self._r1))
            print("r2: " + str(self._r2))
            print("r3: " + str(self._r3))
        
        
    def rank0(self, i):
        return i + Reg(1) - self.rank1(i)
    
    def select0(self, i):
        pass
    
    def rank1(self, i):
        bs1num = i / self.bs[Reg(1)]
        bs1start = bs1num * self.bs[Reg(1)]
        bs1end = bs1start + self.bs[Reg(1)]
        i -= bs1start
        bs2num = i / self.bs[Reg(2)]
        bs2start = bs2num * self.bs[Reg(2)]
        bs2end = bs2start + self.bs[Reg(2)]
        bs2v = self._bv._getMultiBits(bs1start + bs2start, bs1start + bs2end)
        bs2i = i - bs2start
        
        rank = Reg(0)
        rank += self._r1[bs1num]
        rank += self._r2[bs1num * (self.bs[Reg(1)].divceil(self.bs[Reg(2)])) + bs2num]
        rank += self._r3[bs2v * self.bs[Reg(2)] + bs2i]
        return rank
    
    def select1(self, i):
        pass
    
    def rank0BF(self, i):
        return i + 1 - self.rank1BF(i)
    
    def select0BF(self, i):
        return self._s0bf[i]
    
    def rank1BF(self, i):
        return self._r1bf[i]
    
    def select1BF(self, i):
        return self._s1bf[i]
    
    def test(self, iteration):
        for _ in range(iteration):
            i = random.randrange(0, self.n)
            assert self.rank0(Reg(i)).get() == self.rank0BF(i)
            assert self.rank1(Reg(i)).get() == self.rank1BF(i)

In [7]:
if __name__ == '__main__':
    # Test the bit vector
    for n in range(200, 8000, 400):
        print("n =", n)
        time = 0
        space = 0
        spaceBF = 0
        for _ in range(10):
            CreateMachine(math.ceil(math.log(n, 2)) + 1, 16)
            bv = BitVector(n)
            bv._randomize()
            bv.buildIndex()
            GetMachine().totalExecution = 0
            bv.test(200)
            time += GetMachine().totalExecution
            space += GetMachine().totalMemSize
            spaceBF += bv.indexSizeBF
        print("time =", time)
        print("space =", space, space / n)
        print("spaceBF = ", spaceBF)
        print()

n = 200
time = 326000
space = 9600 48.0
spaceBF =  16000

n = 600
time = 342000
space = 20900 34.833333333333336
spaceBF =  60000

n = 1000
time = 342000
space = 30900 30.9
spaceBF =  100000

n = 1400
time = 341972
space = 42330 30.235714285714284
spaceBF =  154000

n = 1800
time = 342000
space = 51910 28.83888888888889
spaceBF =  198000

n = 2200
time = 357960
space = 67480 30.672727272727272
spaceBF =  264000

n = 2600
time = 357920
space = 77600 29.846153846153847
spaceBF =  312000

n = 3000
time = 358000
space = 85680 28.56
spaceBF =  360000

n = 3400
time = 357976
space = 95800 28.176470588235293
spaceBF =  408000

n = 3800
time = 357968
space = 105920 27.873684210526317
spaceBF =  456000

n = 4200
time = 358000
space = 116200 27.666666666666668
spaceBF =  546000

n = 4600
time = 358000
space = 127550 27.72826086956522
spaceBF =  598000

n = 5000
time = 357988
space = 136450 27.29
spaceBF =  650000

n = 5400
time = 357996
space = 145350 26.916666666666668
spaceBF =  702000

n = 58